In [86]:
import urllib.request, sys
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import re
from urllib.error import HTTPError


In [87]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}


In [114]:
## Read the csv file with all listings
Edgar_10K_DF = pd.read_csv('./index_files/EDGAR_10K_10Q.csv')

## Rename the column headers
Edgar_10K_DF.columns = ['S.No', 'CompanyID', 'CompanyName', 'TypeOfFiling', 'DateFiled', 'TextURL', 'HTMLURL']
sec_URL = 'https://www.sec.gov/Archives/'

Edgar_10K_DF['TextURL'] = sec_URL + Edgar_10K_DF['TextURL'].astype(str)
Edgar_10K_DF['HTMLURL'] = sec_URL + Edgar_10K_DF['HTMLURL'].astype(str)

## Create output file name to save the extracted text
outputFileNameList = []
for i in range(0, len(Edgar_10K_DF)):
    outputFileNameList.append((Edgar_10K_DF['CompanyName'][i]).replace("\\","_").replace("/","_").replace(" ","_").replace(".","").replace(",",""))

Edgar_10K_DF['TextOutputFileName'] = outputFileNameList

a = Edgar_10K_DF.head(10)
Edgar_10K_DF = a


,S.No,CompanyID,CompanyName,TypeOfFiling,DateFiled,TextURL,HTMLURL,TextOutputFileName
0,69,1000209,MEDALLION FINANCIAL CORP,10-K,2018-03-14,https://www.sec.gov/Archives/edgar/data/100020...,https://www.sec.gov/Archives/edgar/data/100020...,MEDALLION_FINANCIAL_CORP
1,93,1000228,HENRY SCHEIN INC,10-K,2018-02-21,https://www.sec.gov/Archives/edgar/data/100022...,https://www.sec.gov/Archives/edgar/data/100022...,HENRY_SCHEIN_INC
2,145,1000229,CORE LABORATORIES N V,10-K,2018-02-12,https://www.sec.gov/Archives/edgar/data/100022...,https://www.sec.gov/Archives/edgar/data/100022...,CORE_LABORATORIES_N_V
3,180,1000232,KENTUCKY BANCSHARES INC /KY/,10-K,2018-03-16,https://www.sec.gov/Archives/edgar/data/100023...,https://www.sec.gov/Archives/edgar/data/100023...,KENTUCKY_BANCSHARES_INC__KY_
4,633,1000298,IMPAC MORTGAGE HOLDINGS INC,10-K,2018-03-16,https://www.sec.gov/Archives/edgar/data/100029...,https://www.sec.gov/Archives/edgar/data/100029...,IMPAC_MORTGAGE_HOLDINGS_INC
5,651,1000623,SCHWEITZER MAUDUIT INTERNATIONAL INC,10-K,2018-03-01,https://www.sec.gov/Archives/edgar/data/100062...,https://www.sec.gov/Archives/edgar/data/100062...,SCHWEITZER_MAUDUIT_INTERNATIONAL_INC
6,690,1000694,NOVAVAX INC,10-K,2018-03-14,https://www.sec.gov/Archives/edgar/data/100069...,https://www.sec.gov/Archives/edgar/data/100069...,NOVAVAX_INC
7,701,1000697,WATERS CORP /DE/,10-K,2018-02-27,https://www.sec.gov/Archives/edgar/data/100069...,https://www.sec.gov/Archives/edgar/data/100069...,WATERS_CORP__DE_
8,740,1000753,"INSPERITY, INC.",10-K,2018-02-12,https://www.sec.gov/Archives/edgar/data/100075...,https://www.sec.gov/Archives/edgar/data/100075...,INSPERITY_INC
9,808,1001082,DISH Network CORP,10-K,2018-02-21,https://www.sec.gov/Archives/edgar/data/100108...,https://www.sec.gov/Archives/edgar/data/100108...,DISH_Network_CORP


In [115]:
## Extract the actual 10-K html from index page of SEC filings
main_url = 'https://www.sec.gov'
Html_10K_url = []
for url in Edgar_10K_DF['HTMLURL']:
    pageReq = urllib.request.Request(url, headers = hdr) 
    page = urllib.request.urlopen(pageReq).read()
    soup = BeautifulSoup(page,'lxml')

    for td in soup.find_all('td'):
        if(td.text == '10-K'):
            for a in td.previous_sibling.previous_sibling.find_all('a', href=True):
                #Create a newlist with actual 10-K html url
                Html_10K_url.append(main_url + a['href'])

                
Edgar_10K_DF['Html_10K_url'] = Html_10K_url


In [116]:
## Extract the text from html
for i in range(0, len(Edgar_10K_DF)):
    ## Open the html url
    pageReq = urllib.request.Request(Edgar_10K_DF['Html_10K_url'][i], headers=hdr) 
    page = urllib.request.urlopen(pageReq).read()

    soup = BeautifulSoup(page,'lxml')
    p_content = ''

    ## Remove all the tables
    for table in soup.find_all('table'):
        table.decompose()        


    ## Remove all the table headers
    for a in soup.find_all('a', href=True):
        if a['href'] == '#TABLEOFCONTENTS' or a['href'] == '#Toc':
            a.decompose()


    ## Remove the footers
    for footer in soup.find_all('div', bclfooter = True):
        footer.decompose()

    ## Extract the text under p tags
    for elem in soup.find_all('p'):
        p_content = p_content + (elem.text)

    ## Remove all the white spaces
    stringExtracted = str(p_content)
    stringExtracted = re.sub('\s+', ' ',stringExtracted)
    
    print(Edgar_10K_DF['TextOutputFileName'][i])
    ## Save the extracted tex to corresponding file
    filename = './10KQ_TextFiles/' + Edgar_10K_DF['TextOutputFileName'][i] + '.txt'
    with open(filename,'w', encoding='utf-8') as file:
        file.write(stringExtracted)


MEDALLION_FINANCIAL_CORP
HENRY_SCHEIN_INC
CORE_LABORATORIES_N_V
KENTUCKY_BANCSHARES_INC__KY_
IMPAC_MORTGAGE_HOLDINGS_INC
SCHWEITZER_MAUDUIT_INTERNATIONAL_INC
NOVAVAX_INC
WATERS_CORP__DE_
INSPERITY_INC
DISH_Network_CORP
